# German Credit Scoring Feed

This notebook can be scheduled to provide a continuous flow of data to your German Credit Risk model. Paste your Watson Machine Learning credentials below, and verify that the value of the **DEPLOYMENT_NAME** variable matches the name of your deployed German Credit Risk model. You can also set the maximum and minimum number of records that will be scored per run.

In [1]:
!pip install --upgrade watson-machine-learning-client --no-cache | tail -n 1

In [2]:
WML_CREDENTIALS = {}

In [3]:
# The code was removed by Watson Studio for sharing.

In [4]:
DEPLOYMENT_NAME = "GermanCreditRiskModel"
MIN_RECORDS = 30
MAX_RECORDS = 200

In [5]:
import json, random
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [6]:
wml_client = WatsonMachineLearningAPIClient(WML_CREDENTIALS)

In [7]:
wml_deployments = wml_client.deployments.get_details()
scoring_url = None
for deployment in wml_deployments['resources']:
    if DEPLOYMENT_NAME == deployment['entity']['name']:
        scoring_url = deployment['entity']['scoring_url']
        break
    
print("Scoring URL: {}".format(scoring_url))

Scoring URL: https://us-south.ml.cloud.ibm.com/v3/wml_instances/588024b3-f0f5-489c-8677-a084c8407249/deployments/b68b996f-dea6-4fe7-b6c7-c9a101b4cdf7/online


In [8]:
try:
    with open('german_credit_feed.json', 'r') as scoring_file:
        scoring_data = json.load(scoring_file)
    print('file found')
    
except:
    !wget https://raw.githubusercontent.com/emartensibm/german-credit/master/german_credit_feed.json
    with open('german_credit_feed.json', 'r') as scoring_file:
        scoring_data = json.load(scoring_file)
    print('file downloaded')

--2019-07-22 10:43:45--  https://raw.githubusercontent.com/emartensibm/german-credit/master/german_credit_feed.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3076547 (2.9M) [text/plain]
Saving to: ‘german_credit_feed.json’

100%[======================================>] 3,076,547   --.-K/s   in 0.09s   

2019-07-22 10:43:46 (32.9 MB/s) - ‘german_credit_feed.json’ saved [3076547/3076547]

file downloaded


In [9]:
fields = scoring_data['fields']
values = []
for _ in range(0, random.randint(MIN_RECORDS, MAX_RECORDS)):
    values.append(random.choice(scoring_data['values']))
payload_scoring = {"fields": fields, "values": values}

scoring_response = wml_client.deployments.score(scoring_url, payload_scoring)
print(scoring_response)

{'fields': ['CheckingStatus', 'LoanDuration', 'CreditHistory', 'LoanPurpose', 'LoanAmount', 'ExistingSavings', 'EmploymentDuration', 'InstallmentPercent', 'Sex', 'OthersOnLoan', 'CurrentResidenceDuration', 'OwnsProperty', 'Age', 'InstallmentPlans', 'Housing', 'ExistingCreditsCount', 'Job', 'Dependents', 'Telephone', 'ForeignWorker', 'CheckingStatus_IX', 'CreditHistory_IX', 'EmploymentDuration_IX', 'ExistingSavings_IX', 'ForeignWorker_IX', 'Housing_IX', 'InstallmentPlans_IX', 'Job_IX', 'LoanPurpose_IX', 'OthersOnLoan_IX', 'OwnsProperty_IX', 'Sex_IX', 'Telephone_IX', 'features', 'rawPrediction', 'probability', 'prediction', 'predictedLabel'], 'values': [['0_to_200', 14, 'credits_paid_to_date', 'furniture', 250, 'less_100', 'less_1', 2, 'female', 'none', 3, 'real_estate', 19, 'bank', 'own', 1, 'unskilled', 1, 'none', 'yes', 2.0, 1.0, 3.0, 0.0, 0.0, 0.0, 2.0, 1.0, 1.0, 0.0, 2.0, 1.0, 0.0, [2.0, 1.0, 1.0, 0.0, 3.0, 1.0, 0.0, 2.0, 2.0, 0.0, 1.0, 0.0, 0.0, 14.0, 250.0, 2.0, 3.0, 14.0, 19.0, 1